In [26]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from datetime import date
from datetime import timedelta
from ta.volatility import BollingerBands, AverageTrueRange
import math

In [27]:
Features = pd.read_excel("Features.xlsx", index_col = 0)

In [28]:
Features

,Date,AD_Ratio,5%_Up,8%_Up,13%_Up,Down,Frac_EMA8,Frac_EMA13,Frac_EMA21,Frac_EMA34,...,PCT_Change_Nifty,Nifty_EMA8,Nifty_EMA13,Nifty_EMA21,Nifty_EMA34,Nifty_EMA55,Nifty_EMA89,Nifty_EMA144,Nifty_EMA233,Labels
0,2010-08-02,2.428571,0.125000,0.083333,0.041667,7,0.833333,0.875000,0.875000,0.875000,...,0.013349,0,0,0,0,0,0,0,0,2
1,2010-08-09,7.000000,0.291667,0.166667,0.041667,3,0.875000,0.916667,0.916667,0.916667,...,0.002362,1,1,1,1,1,1,1,1,2
2,2010-08-16,3.800000,0.375000,0.250000,0.041667,5,0.916667,0.916667,0.916667,0.916667,...,0.014407,1,1,1,1,1,1,1,1,2
3,2010-08-23,0.200000,0.041667,0.041667,0.041667,20,0.875000,0.916667,0.916667,0.916667,...,-0.022050,1,1,1,1,1,1,1,1,0
4,2010-08-30,5.000000,0.250000,0.041667,0.083333,4,0.916667,0.916667,0.916667,0.916667,...,0.013071,0,0,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2021-07-12,1.000000,0.041667,0.041667,0.041667,12,0.541667,0.625000,0.750000,0.833333,...,0.014889,1,1,1,1,1,1,1,1,2
572,2021-07-19,0.263158,0.041667,0.041667,0.041667,19,0.250000,0.291667,0.625000,0.750000,...,-0.004230,1,1,1,1,1,1,1,1,0
573,2021-07-26,1.000000,0.041667,0.041667,0.041667,12,0.333333,0.500000,0.625000,0.708333,...,-0.005865,1,1,1,1,1,1,1,1,1
574,2021-08-02,0.846154,0.083333,0.041667,0.041667,13,0.416667,0.416667,0.625000,0.708333,...,0.030143,1,1,1,1,1,1,1,1,2


In [29]:
def Add_Indicators(Data, CMIE):
    
    BB = BollingerBands(close = Data["Bank_Nifty"], window = 5, window_dev= 2, fillna = True)
    AVG = AverageTrueRange(high = CMIE["High"], close = CMIE["Close"], low = CMIE["Low"], window = 4, fillna = True)
    
    H_BB = BB.bollinger_hband()
    L_BB = BB.bollinger_lband()
    M_BB = BB.bollinger_mavg()
    ATR  = AVG.average_true_range()
    
    return(H_BB, L_BB, M_BB, ATR)

In [30]:
CMIE = pd.read_excel("New_CMIE_Bank_Nifty.xlsx", index_col = 0)

In [31]:
CMIE

,Low,High,Close
Date,,,
2010-08-07,10364.95,10476.70,10383.65
2010-08-14,10603.90,10782.20,10737.35
2010-08-21,11005.50,11105.00,11033.75
2010-08-28,10681.60,11003.25,10727.45
2010-09-04,10938.90,11017.15,10991.20
...,...,...,...
2021-06-12,34891.30,35344.00,35047.40
2021-06-19,33908.95,34787.40,34558.00
2021-06-26,34915.85,35491.95,35364.65


In [32]:
start_date = date(2010,1,1)
today = datetime.datetime.today()

In [33]:
New_CMIE = yf.download("^NSEBANK", start= start_date, end = today, interval= "1wk", threads= True, group_by= "Ticker")

[*********************100%***********************]  1 of 1 completed


In [34]:
New_CMIE = New_CMIE[New_CMIE.index > CMIE.index[-1]]

In [35]:
New_CMIE

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-12,35285.050781,35985.250000,35060.351562,35751.800781,35751.800781,0
2021-07-19,35070.500000,35347.050781,34357.351562,35034.398438,35034.398438,0
2021-07-26,34947.851562,35186.101562,34115.199219,34584.351562,34584.351562,0
2021-08-02,34760.898438,36219.800781,34588.250000,35809.250000,35809.250000,0
2021-08-06,35875.800781,36080.101562,35700.449219,35809.250000,35809.250000,0


In [36]:
#for i in range(New_CMIE.shape[0]):
#    new_date = np.append(new_date, New_CMIE.Date.iloc[-1] + (i+1) * timedelta(days = 7))
#    new_attribute = np.append(new_attribute, "Index Closing")
#    new_closing  = np.append(new_closing, New_CMIE.Close.iloc[i])
#
#new_row =pd.DataFrame({"Date": new_date, "Attribute": new_attribute, "Closing": new_closing})
#CMIE = Bank_Nifty.append(new_row, ignore_index = True, sort = False)

In [37]:
CMIE = CMIE.append(New_CMIE, ignore_index= False, sort = False)

In [38]:
H_BB, L_BB, M_BB, ATR = Add_Indicators(Features, CMIE)

In [39]:
Features.shape

(576, 28)

In [40]:
CMIE.shape

(576, 6)

In [41]:
if(today.day!= 7 or today.day!= 0 ):
    Features["ATR"] = np.array(ATR*100/(CMIE["High"]- CMIE["Low"]))[:-1]
else:
    Features["ATR"] = np.array(ATR*100/(CMIE["High"]- CMIE["Low"]))

ValueError: Length of values (575) does not match length of index (576)

In [ ]:
Features.columns

Index(['Date', 'AD_Ratio', '5%_Up', '8%_Up', '13%_Up', 'Down', 'Frac_EMA8',
       'Frac_EMA13', 'Frac_EMA21', 'Frac_EMA34', 'Frac_EMA55', 'Frac_EMA89',
       'Frac_EMA144', 'Frac_EMA233', 'India_Vix', 'PCT_Change_VIX',
       'Bank_Nifty', 'PCT_Change_Bank_Nifty', 'PCT_Change_Nifty', 'Nifty_EMA8',
       'Nifty_EMA13', 'Nifty_EMA21', 'Nifty_EMA34', 'Nifty_EMA55',
       'Nifty_EMA89', 'Nifty_EMA144', 'Nifty_EMA233', 'Labels', 'ATR'],
      dtype='object')

In [42]:
Features["L_BB_PCT_CHANGE"] = (Features["Bank_Nifty"] - L_BB)*100/Features["Bank_Nifty"]
Features["H_BB_PCT_CHANGE"] = (Features["Bank_Nifty"] - H_BB)*100/Features["Bank_Nifty"]
Features["B_BB_PCT_CHANGE"] = (Features["Bank_Nifty"] - M_BB)*100/Features["Bank_Nifty"]

In [43]:
Features.head()

,Date,AD_Ratio,5%_Up,8%_Up,13%_Up,Down,Frac_EMA8,Frac_EMA13,Frac_EMA21,Frac_EMA34,...,Nifty_EMA21,Nifty_EMA34,Nifty_EMA55,Nifty_EMA89,Nifty_EMA144,Nifty_EMA233,Labels,L_BB_PCT_CHANGE,H_BB_PCT_CHANGE,B_BB_PCT_CHANGE
0,2010-08-02,2.428571,0.125000,0.083333,0.041667,7,0.833333,0.875000,0.875000,0.875000,...,0,0,0,0,0,0,2,0.000000,0.000000,0.000000
1,2010-08-09,7.000000,0.291667,0.166667,0.041667,3,0.875000,0.916667,0.916667,0.916667,...,1,1,1,1,1,1,2,4.941163,-1.647054,1.647054
2,2010-08-16,3.800000,0.375000,0.250000,0.041667,5,0.916667,0.916667,0.916667,0.916667,...,1,1,1,1,1,1,2,7.676368,-1.957551,2.859409
3,2010-08-23,0.200000,0.041667,0.041667,0.041667,20,0.875000,0.916667,0.916667,0.916667,...,1,1,1,1,1,1,0,4.355685,-4.227043,0.064321
4,2010-08-30,5.000000,0.250000,0.041667,0.083333,4,0.916667,0.916667,0.916667,0.916667,...,1,1,1,1,1,1,2,6.202518,-2.262639,1.969940


In [44]:
Features["Lagged_Labels"] = Features["Labels"]
Features["Labels"] = Features["Labels"].shift(-1)
Features = Features.reset_index(drop = True)

In [45]:
Features.columns

Index(['Date', 'AD_Ratio', '5%_Up', '8%_Up', '13%_Up', 'Down', 'Frac_EMA8',
       'Frac_EMA13', 'Frac_EMA21', 'Frac_EMA34', 'Frac_EMA55', 'Frac_EMA89',
       'Frac_EMA144', 'Frac_EMA233', 'India_Vix', 'PCT_Change_VIX',
       'Bank_Nifty', 'PCT_Change_Bank_Nifty', 'PCT_Change_Nifty', 'Nifty_EMA8',
       'Nifty_EMA13', 'Nifty_EMA21', 'Nifty_EMA34', 'Nifty_EMA55',
       'Nifty_EMA89', 'Nifty_EMA144', 'Nifty_EMA233', 'Labels',
       'L_BB_PCT_CHANGE', 'H_BB_PCT_CHANGE', 'B_BB_PCT_CHANGE',
       'Lagged_Labels'],
      dtype='object')

In [46]:
Labels = Features["Labels"]
Features = Features.drop("Labels", axis =1)
Features["Labels"] = Labels

In [47]:
Features.columns

Index(['Date', 'AD_Ratio', '5%_Up', '8%_Up', '13%_Up', 'Down', 'Frac_EMA8',
       'Frac_EMA13', 'Frac_EMA21', 'Frac_EMA34', 'Frac_EMA55', 'Frac_EMA89',
       'Frac_EMA144', 'Frac_EMA233', 'India_Vix', 'PCT_Change_VIX',
       'Bank_Nifty', 'PCT_Change_Bank_Nifty', 'PCT_Change_Nifty', 'Nifty_EMA8',
       'Nifty_EMA13', 'Nifty_EMA21', 'Nifty_EMA34', 'Nifty_EMA55',
       'Nifty_EMA89', 'Nifty_EMA144', 'Nifty_EMA233', 'L_BB_PCT_CHANGE',
       'H_BB_PCT_CHANGE', 'B_BB_PCT_CHANGE', 'Lagged_Labels', 'Labels'],
      dtype='object')

In [48]:
#Add a dummy variable
Features["Labels"].iloc[-1]  = 0 

C:\Users\viraj\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [50]:
Features.to_excel("Features_Final.xlsx")